In [1]:
using Plots
pyplot()
using ColoringNames
using CSVFiles
using DataFrames
pyplot()

runnum = "0a"
function mklogdir(name)
    logdir = joinpath(pwd(), "logs","point", name * runnum)
    mkpath(logdir)
    logdir
end

mklogdir (generic function with 1 method)

In [2]:
const many_names = load_color_nameset()
const word_embs, vocab, enc = load_text_embeddings(keep_words=many_names)
const full_data = load_munroe_data(dev_as_train=false, dev_as_test=false, encoding_ = enc)

const ord_data = order_relevant_dataset(full_data);
const extrapo_data = extrapolation_dataset(full_data);

In [3]:
function get_saveplot(mdlname)
    figpath = joinpath("demo", "point", mdlname)
    mkpath(figpath)
    function saveplot(mdl, colornames, subfigname)
        filename = joinpath(figpath, subfigname*".png")
        savefig(plot_query(mdl, colornames), filename)
    end
end


function qualitative_demo(mdl, do_oov=true;
        demofun= (mdl, colornames, subfigname) -> display(plot_query(mdl, colornames))
    )
    
    maincolors = [   "brown-orange",
                "orange-brown",
                "yellow-orange",
                "orange-yellow",
                "brownish green",
                "greenish brown",
                "bluish grey",
                "greyish blue",
                "pink-purple",
                "purple-pink",
                "green",
                "greenish",
                "purple",
                "purplish",
                "brown",
                "brownish",
                "black",
                "white",
                "grey"]
    demofun(mdl, maincolors, "maincolors")
    
    if do_oov
        oov_names = ["Brown", "Green", "Purple", "gray", "Gray"]
        #append!(names, ["ish"])
        demofun(mdl, oov_names, "oovcolors")
    end
    
end

qualitative_demo(mdl::TermToColorPointEmpirical; kwargs...) = qualitative_demo(mdl, false; kwargs...)

qualitative_demo (generic function with 3 methods)

In [4]:
create_res_df() = DataFrame(method=String[], mse=Float64[])

full_df = create_res_df()
extrapo_df = create_res_df()
ord_df = create_res_df()

function perform_evaluation(modeltype, name)
    info(name)
    
    function mdlfun(cldata)
        mdl = modeltype(enc, word_embs; n_steps=size(cldata.train.terms_padded,1))
        train!(mdl, cldata; log_dir = mklogdir(name))
        mdl
    end
    
    
    ###################
    if doextrapo(modeltype)
        extrapo_mdl = mdlfun(extrapo_data)
        println()
        @show res_extrapo = evaluate(extrapo_mdl, extrapo_data.test)
        push!(extrapo_df, Dict(:method=>name * " Extrapolating", :mse=>res_extrapo))
    end
    
    
    ######
    full_mdl = mdlfun(full_data)
    println()
    @show res_full = evaluate(full_mdl, full_data.test)
    push!(full_df, Dict(:method=>name, :mse=>res_full))
    
    qualitative_demo(full_mdl; demofun=get_saveplot(name))
    
    #######
    println()
    @show res_ord = evaluate(full_mdl, ord_data.test)
    push!(ord_df, Dict(:method=>name, :mse=>res_ord))
    
    
    #######
    @show  res_nonextrapo = evaluate(full_mdl, extrapo_data.test)
    println()
    push!(extrapo_df, Dict(:method=>name * " Non-extrapolating", :mse=>res_nonextrapo))
    
    
    ####
    save("results/raw/point_full.csv", full_df)
    save("results/raw/point_extrapo.csv", extrapo_df)
    save("results/raw/point_ord.csv", ord_df)
    
    full_mdl
end


doextrapo(::Any) = true
doextrapo(::Type{TermToColorPointEmpirical}) = false

doextrapo (generic function with 2 methods)

In [5]:
namedmodels = [
  #  (TermToColorPointEmpirical, "Direct"),
   # (TermToColorPointSOWE, "SOWE"),
    #(TermToColorPointCNN, "CNN"),
    (TermToColorPointRNN, "LSTM"),    
]

for (modeltype, name) in namedmodels
    perform_evaluation(modeltype, name)
end;

INFO: LSTM
2019-01-25 13:05:57.196061: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.2 AVX AVX2 FMA



res_extrapo = evaluate(extrapo_mdl, extrapo_data.test) = 0.14087577f0



res_full = evaluate(full_mdl, full_data.test) = 0.071216844f0


'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
/Users/oxinabox/.julia/v0.6/Conda/deps/usr/lib/python3.6/site-packages/matplotlib/cbook/__init__.py:424: MatplotlibDeprecationWarning: 
Passing one of 'on', 'true', 'off', 'false' as a boolean is deprecated; use an actual boolean (True/False) instead.
  warn_deprecated("2.2", "Passing one of 'on', 'true', 'off', 'false' as a "
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.



res_ord = evaluate(full_mdl, ord_data.test) = 0.096134596f0
res_nonextrapo = evaluate(full_mdl, extrapo_data.test) = 0.13809597f0



In [7]:
1

1

In [ ]:
#noml = ColoringNames.TermToColorPointEmpirical()
#train!(noml, full_data)

qualitative_demo(noml, demofun=get_saveplot("Direct"))


In [ ]:
noml = ColoringNames.TermToColorPointEmpirical()
train!(noml, cldata)

qualitative_demo(noml)
@show evaluate(noml, cldata.dev)

In [ ]:
sowe = TermToColorPointSOWE(full_data.encoding, word_embs; n_steps=4)
train!(sowe, full_data;
    log_dir=mklogdir("sowe")
)
qualitative_demo(sowe)
@show evaluate(sowe, full_data.dev)

In [ ]:
cnn = TermToColorPointCNN(cldata.encoding, word_embs; n_steps=4)

train!(cnn, cldata;
    log_dir=mklogdir("cnn"),
)
qualitative_demo(cnn)
@show evaluate(cnn, cldata.dev)

In [ ]:
qualitative_demo(cnn)
@show evaluate(cnn, cldata.dev)

In [ ]:
rnn = TermToColorPointRNN(cldata.encoding, word_embs; n_steps=4)

train!(rnn, cldata;
    log_dir=mklogdir("rnn"),
)
qualitative_demo(rnn)
@show evaluate(rnn, cldata.dev)

In [ ]:
1

In [ ]:
using ColoringNames: order_relevant_dataset, order_relevant_name_pairs

In [ ]:
namepairs = order_relevant_name_pairs(cldata.dev);

In [ ]:
hsv1s = Array{Float64}((length(namepairs), 3))
hsv2s = similar(hsv1s)
for (ii, (n1, n2)) in enumerate(namepairs)
    hsv1s[ii, :] = query(noml, [n1])
    hsv2s[ii, :] = query(noml, [n2])
end
selectperm(ColoringNames.hsv_squared_error(hsv1s, hsv2s), 1:5)

In [ ]:
namepairs[[12, 15, 13, 14, 16]]

In [ ]:
plot_query(mdl, "pink-purple") |> display
plot_query(mdl, "purple-pink") |> display
plot_query(mdl, "brown-orange") |> display
plot_query(mdl, "orange-brown") |> display
plot_query(mdl, "orange-yellow") |> display
plot_query(mdl, "yellow-orange") |> display
plot_query(mdl, "orange-yellow") |> display
plot_query(mdl, "yellow-orange") |> display

In [ ]:
'-' .∈ collect(names) |> any